In [ ]:
import cv2
import mediapipe as mp
import numpy as np

In [ ]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [ ]:
import csv
import os
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
landmarks = ['class']
for val in range(1, 33+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [ ]:
with open('coords.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [ ]:
with open('coords_goodmorning.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [ ]:
def export_landmark(results, action):
    try:
        keypoints = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist()
        keypoints.insert(0,action)
        
        print("Upload Succesfull", action)
        
        with open('coords.csv', mode='a', newline='') as f:
            csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow(keypoints)
    except Exception as e:
        print("Error:", e)
        pass


In [ ]:
def export_landmark(results, action):
    try:
        keypoints = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist()
        keypoints.insert(0,action)
        
        print("Upload Succesfull", action)
        
        with open('coords_goodmorning.csv', mode='a', newline='') as f:
            csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow(keypoints)
    except Exception as e:
        print("Error:", e)
        pass


# Labelling

In [ ]:
with open('last_processed_index.pkl', 'rb') as file:
    my_object = pickle.load(file)
    print(my_object)

In [ ]:
import os
import pickle

# Load or initialize the last processed video index
last_processed_index_file = 'last_processed_index.pkl'
if os.path.isfile(last_processed_index_file):
    with open(last_processed_index_file, 'rb') as f:
        last_processed_index = pickle.load(f)
else:
    last_processed_index = 0

for x in range(last_processed_index + 1, 6):
    video_path = r"C:\Users\eddcr\Downloads\Wrong Squat\good morning backlight\squat ({}).mp4".format(x)
    
    # Check if the video file exists before processing
    if not os.path.isfile(video_path):
        print(f"Video file not found: {video_path}")
        continue

    cap = cv2.VideoCapture(video_path)
    pause = False  # Flag to indicate whether the video is paused

    # initiate holistic model
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            ret, frame = cap.read()

            if not ret:
                break

            # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            # Make Detection
            results = pose.process(image)

            # Recolor image to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            # Render detections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            # Add text overlay to show the currently playing video
            cv2.putText(image, f"Video: squat({x})", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            
            k = cv2.waitKey(100)
            if k == 117:  # press u to capture upwards position
                export_landmark(results, 'up')
            if k == 100:  # press d to capture downwards position
                export_landmark(results, 'down')
            if k == 112:  # press p to toggle pause/play
                pause = not pause
            if k == 27:  # press Esc key to exit
                break
            
            if not pause:
                display_image = cv2.resize(image, (640, 480))
                cv2.imshow('Mediapipe Feed', display_image)
                
        cap.release()
        cv2.destroyAllWindows()
    # Save the last processed index
    with open(last_processed_index_file, 'wb') as f:
        pickle.dump(x, f)
        
    if cv2.waitKey(0) & 0xFF == ord('p'):
            # If 'p' is pressed after a video, pause before proceeding to the next video
            pause = True
            
    exit_choice = input("Press 'q' to quit or any other key to continue to the next video: ")
    if exit_choice.lower() == 'q':
        break

# Training

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.model_selection import cross_val_score, KFold

In [ ]:
df = pd.read_csv('coords.csv')
x = df.drop('class', axis = 1)
y = df['class']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.3, random_state = 2)

y_test

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [ ]:
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
    'svm': make_pipeline(StandardScaler(), SVC(kernel='linear', C=1.0))
}

In [ ]:
from sklearn.multiclass import OneVsRestClassifier

pipelines = {
    'lr': make_pipeline(StandardScaler(), LogisticRegression(multi_class='ovr')),
    'rc': make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf': make_pipeline(StandardScaler(), RandomForestClassifier())
}


In [ ]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(x_train, y_train)
    fit_models[algo] = model

In [ ]:
# Specify the number of folds (e.g., 5-fold cross-validation)
num_folds = 10
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Loop through each model in the pipelines and perform cross-validation
for algo, pipeline in pipelines.items():
    scores = cross_val_score(pipeline, x_train, y_train, cv=kf, scoring='accuracy')
    print(f'{algo} Cross-Validation Scores: {scores}')
    print(f'{algo} Mean Accuracy: {scores.mean()}')
    print()

In [ ]:
fit_models

In [ ]:
fit_models['rf'].predict(x_test)

# Evaluate

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
import pickle

In [ ]:
for algo, model in fit_models.items():
    yhat = model.predict(x_test)
    print(algo, accuracy_score(y_test.values, yhat), 
        precision_score(y_test.values, yhat, average = "binary", pos_label="down"),
        recall_score(y_test.values, yhat, average="binary", pos_label="down"))

In [ ]:
for algo, model in fit_models.items():
    yhat = model.predict(x_test)
    print(algo, accuracy_score(y_test, yhat), 
          precision_score(y_test, yhat, average='weighted', labels=np.unique(yhat)),
          recall_score(y_test, yhat, average='weighted', labels=np.unique(yhat)))

In [ ]:
yhat = fit_models['rf'].predict(x_test)

In [ ]:
with open('squattest.pkl', 'wb') as f:
    pickle.dump(fit_models['rf'], f)

# Detect Using Model

In [ ]:
import pickle

with open('squattest.pkl', 'rb') as f:
    model = pickle.load(f)

In [ ]:
with open('squattest.pkl', 'rb') as file:
    my_object = pickle.load(file)
    print(my_object)

In [ ]:
import cv2

for i in range(10):
    cap = cv2.VideoCapture(i)
    if cap.isOpened():
        print(f"Camera index {i} is working.")
        cap.set(3, 640)  # Width
        cap.set(4, 480)  # Height
        cap.release()
    else:
        print(f"Camera index {i} is not working.")


In [ ]:
cap = cv2.VideoCapture(0)
counter = 0
current_stage = ''

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        #cap.set(3, 640)  # Width
        #cap.set(4, 480)  # Height

        #Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        #Make Detection
        results = pose.process(image)

        #Recolor image to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
        
        #image = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
       
        #Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        
        try:
            row = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist()
            X = pd.DataFrame([row], columns=landmarks[1:])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            print(body_language_class, body_language_prob)
            
            # counter
            if body_language_class == 'up' and body_language_prob[body_language_prob.argmax()] >= .7:
                current_stage = 'up'
            elif current_stage == 'up' and body_language_class == 'down' and body_language_prob[body_language_prob.argmax()] >= .7:
                current_stage="down"
                counter +=1
                print(current_stage)
            
            
            
        except Exception as e:
            print("Error")
            pass
        
        #Get status box
        cv2.rectangle (image, (0,0), (250, 60), (245, 117, 16), -1)

        #Display Class
        cv2.putText(image, 'CLASS'
                    , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, body_language_class.split(' ')[0]
                    , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,255), 2, cv2.LINE_AA)

        #display probability
        cv2.putText(image, 'PROB'
                    , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                    , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,255), 2, cv2.LINE_AA)

        #display counter
        cv2.putText(image, 'COUNT'
                    , (180,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter)
                    , (175,40), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,255), 2, cv2.LINE_AA)
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord ('q'):
            break
        
cap.release()
cv2.destroyAllWindows()